In [1]:
try: import simplejson as json
except ImportError: import json

import gzip,codecs,numpy as np,random,copy
import scipy.optimize as opt

load the split dataset into train and test.
build the Iu and Ui data structure

In [106]:
#with open("finefood_train_random.json","r") as infile:
#with open("beeradvocate_train_random.json","r") as infile:
#with open("beeradvocate_train_lastrating.json","r") as infile:
#with open("finefood_train_lastrating.json","r") as infile:
#with open("ratebeer_train_lastrating.json","r") as infile:
with open("ratebeer_train_random.json","r") as infile:
    train = json.load(infile)
infile.close()
#with open("finefood_test_random.json","r") as infile:
#with open("beeradvocate_test_random.json","r") as infile:
#with open("beeradvocate_test_lastrating.json","r") as infile:
#with open("finefood_test_lastrating.json","r") as infile:
#with open("ratebeer_test_lastrating.json","r") as infile:
with open("ratebeer_test_random.json","r") as infile:
    test = json.load(infile)
infile.close()
#with open("finefood_quickmap_random.json","r") as infile:
#with open("beeradvocate_quickmap_random.json","r") as infile:
#with open("beeradvocate_quickmap_lastrating.json","r") as infile:
#with open("finefood_quickmap_lastrating.json","r") as infile:
#with open("ratebeer_quickmap_lastrating.json","r") as infile:
with open("ratebeer_quickmap_random.json","r") as infile:
    quickmap = json.load(infile)
infile.close()

print(len(train),len(test),len(quickmap))
train = sorted(train, key = lambda k : int(k["review/time"]))

2704815 4760 2709575


In [107]:
Iu = dict() #set of products reviewed by users
Ui = dict() #set of users who reviewed the product

In [108]:
for review in train:
        item = review["product/productId"]
        user = review["review/userId"]
        if item in Ui:
            Ui[item].append(user)
        else:
            Ui[item] = [user]
        if user in Iu:
            Iu[user].append(item)
        else:
            Iu[user] = [item]

### Construct user and item mapping to integer indices

In [109]:
distinct_user_set = set()
distinct_item_set = set()
for review in train:
    if review["review/userId"] not in distinct_user_set:
        distinct_user_set.add(review["review/userId"])
    if review["product/productId"] not in distinct_item_set:
        distinct_item_set.add(review["product/productId"])

In [110]:
print(len(distinct_user_set), len(distinct_item_set))

4760 110032


In [111]:
import sys
sys.setrecursionlimit(20000)

In [112]:
#with open("finefood_user_map_random.json",'r') as infile:
with open("ratebeer_user_map_random.json",'r') as infile:
#with open("ratebeer_user_map_lastrating.json",'r') as infile:
        user_map = json.load(infile)
#print("File {} written".format("ratebeer_user_map_lastrating.json"))
infile.close()
#with open("finefood_item_map_random.json",'r') as infile:
with open("ratebeer_item_map_random.json",'r') as infile:
#with open("ratebeer_item_map_lastrating.json",'r') as infile:
        item_map = json.load(infile)
#print("File {} written".format("ratebeer_item_map_lastrating.json"))
infile.close()

In [113]:
user_map_int = {}
for key in user_map:
    user_map_int[int(key)] = user_map[key]
item_map_int = {}
for key in item_map:
    item_map_int[int(key)] = item_map[key]

## Latent factor model  -standard

In [102]:
class LFM(object):
    ''' LFM class implements the standard latent factor model of collaborative filtering using matrix factorization
    '''
    
    def __init__(self,train_data, Iu_reg, Ui_reg, userproduct_dict,userset,itemset,usermap,itemmap,k,Lambda):
        ''' requires Iu and Ui matrix information, quick mapping of reviews to (user,product) and k for set up
        also sets up k fold training data for k-cross validation
        '''
        
        self.Ntrain = len(train_data)               #Number of training samples
        self.train_data = train_data                #the train_data used
        self.Iu = self.deepish_copy(Iu_reg)         #Iu mapping
        self.Ui = self.deepish_copy(Ui_reg)         #Ui mapping
        self.quickmap = userproduct_dict            #uses key as (userid-itemid) for quick mapping to required review
        self.user_set = userset
        self.item_set = itemset
        
        self.user_map = usermap
        self.item_map = itemmap
            
        #hyperparameters
        self.Lambda= Lambda               #regularization param
        self.k=k                          # number of latent factor dimension (low dimensional repr)
        self.final_param = self.init_theta()  #initialize the current final_param to some value.
        
        
    def init_theta(self):
        ''' Initializes the parameter of the standard model
        flat_theta = <alpha, Bu, Bi, Gu,Gu>
        '''
        flat_theta = []
        #baseline predictors
        self.alpha = 0                    #global offset
        flat_theta.append(self.alpha)
        
        self.Bu = dict()                  #user bias initialized randomly
        self.Bi = dict()                  #item bias initialized randomly
        for i in range(len(self.user_map)):
            self.Bu[self.user_map[i]] =np.random.random(1).item() 
            flat_theta.append(self.Bu[self.user_map[i]])
        for i in range(len(self.item_map)):
            self.Bi[self.item_map[i]] = np.random.random(1).item() 
            flat_theta.append(self.Bi[self.item_map[i]])
        
        self.totalparams = 1 + len(self.user_set) + len(self.item_set) 
        return np.array(flat_theta)
    
    
    def retrieve_theta_components(self,theta):
        self.alpha = theta[0]
        j=1
        for i in range(len(self.user_set)):
            self.Bu[self.user_map[i]] = theta[j]
            j+=1
        for i in range(len(self.item_set)):
            self.Bi[self.item_map[i]] = theta[j]
            j+=1
        if j!= len(theta):
            print("Something went wrong. Not all theta values were used")
        
    def pred(self,user,item):
        ''' calculates the current prediction by the model for the rating of user for the given item'''
        return self.alpha + self.Bu[user] + self.Bi[item]
    
    def f(self,theta):
        '''Calculates the value of the objective function (loss) on the training data. 
            Note that the training error is not MSE '''
        #retrieve the individual components of theta
        self.retrieve_theta_components(theta)
        error = 0
        for review in self.train_data:
            user = review['review/userId']
            item = review["product/productId"]
            error += (self.pred(user,item) - review["review/score"])**2
        error/=self.Ntrain
        #regularization terms 
        Bu_np = np.array(list(self.Bu.values()))
        Bi_np = np.array(list(self.Bi.values()))
        reg_complexity = np.sum(np.square(Bu_np)) + np.sum(np.square(Bi_np))
    
        total_mse = error + self.Lambda * reg_complexity
        return total_mse/2.0
    
    def fprime_one_func(self,theta):
        self.retrieve_theta_components(theta)
        flat_gradient = []
        umap_len = len(self.user_map)
        imap_len = len(self.item_map)
        
        self.alpha_grad = 0                              
        self.Bu_grad = dict()                                      
        self.Bi_grad = dict()                                     
          
        for i in range(len(self.user_map)):
            self.Bu_grad[self.user_map[i]] = 0.0
            
        for i in range(len(self.item_map)):
            self.Bi_grad[self.item_map[i]] = 0.0
 
        
        for review in self.train_data:
            user = review["review/userId"]
            item = review["product/productId"]
            delta = self.pred(user,item) - review["review/score"]
            delta /= self.Ntrain
            self.alpha_grad += delta
            self.Bu_grad[user]+= delta
            self.Bi_grad[item]+= delta
        
        for user in self.user_set:
            self.Bu_grad[user] += self.Lambda * self.Bu[user]
            
        for item in self.item_set:
            self.Bi_grad[item] += self.Lambda * self.Bi[item]

        
        flat_gradient.append(self.alpha_grad)
        for i in range(len(self.user_set)):
            flat_gradient.append(self.Bu_grad[self.user_map[i]])
        for i in range(len(self.item_set)):
            flat_gradient.append(self.Bi_grad[self.item_map[i]])

        return np.array(flat_gradient)
    
    def call(self,theta):
        print("{} Objective value: {}".format(self.i, self.f(theta)))
        self.i+=1
    
    def objectiveloss_lbfgs(self,grad_tolerance,fac):
        self.i =0;
        flat_theta_guess = self.final_param  #start with the initial guess or the one made by previous call to func
        flat_theta,value,d = opt.fmin_l_bfgs_b(self.f,flat_theta_guess,self.fprime_one_func,\
                                               pgtol = grad_tolerance,disp=True,\
                                             callback = self.call, iprint=0)
        print("Value of ojective function {} at pgtol={}".format(value,grad_tolerance))
        self.final_param = flat_theta
    
    def mse_test(self,test_data):
        ''' Uses Mean Squared Error as evaluation metric on test data provided by user'''
        self.retrieve_theta_components(self.final_param)
        error = 0
        unknown_data_count =0;
        for review in test_data:
            if review["review/userId"] in self.Bu and review["product/productId"] in self.Bi:
                user = review["review/userId"]
                item = review["product/productId"]
                error += (self.pred(user,item) - review["review/score"] )**2
            else: 
                unknown_data_count+=1
        if unknown_data_count>0:
            print("Warning! Unknown {} new data rows".format(unknown_data_count))
        return error / (len(test_data) - unknown_data_count)
   
    def deepish_copy(self,org):
        '''
        much, much faster than deepcopy, for a dict of the simple python types.
        '''
        out = dict().fromkeys(org)
        for k,v in org.items():
            try:
                out[k] = v.copy()   # dicts, sets
            except AttributeError:
                try:
                    out[k] = v[:]   # lists, tuples, strings, unicode
                except TypeError:
                    out[k] = v      # ints

        return out  

In [114]:
lfmObj = LFM(train,Iu, Ui,quickmap,distinct_user_set,distinct_item_set,user_map_int,item_map_int, 5,0.001)

In [115]:
lfmObj.objectiveloss_lbfgs(1e-5,1e2)

0 Objective value: 20.428455947081567
1 Objective value: 19.50416622149925
2 Objective value: 19.312060726932696
3 Objective value: 16.91199215836161
4 Objective value: 13.142792830633583
5 Objective value: 6.856081699338087
6 Objective value: 2.039378145941633
7 Objective value: 0.5356946428992466
8 Objective value: 0.3948612506757086
9 Objective value: 0.38982840266660845
10 Objective value: 0.37874058054010645
11 Objective value: 0.32546565949127376
12 Objective value: 0.2948140899776007
13 Objective value: 0.2904529011359158
14 Objective value: 0.2898493106914063
15 Objective value: 0.28982081587081693
16 Objective value: 0.28979445922837427
17 Objective value: 0.28945115778729763
18 Objective value: 0.28940231647799824
19 Objective value: 0.28939307504522277
20 Objective value: 0.2893836090171044
21 Objective value: 0.2893740361127268
22 Objective value: 0.2893631230192639
23 Objective value: 0.28935439488209264
24 Objective value: 0.28934601168868257
25 Objective value: 0.2893374

In [116]:
lfmObj.mse_test(test)

Warning! Unknown 13 new data rows


0.66164794509092251

In [118]:
lfmObj.alpha

3.3021826813685182

In [16]:
np.savetxt("ratebeer_standard_lfm_results", lfmObj.final_param)

In [117]:
max(lfmObj.Bi.values())

0.46156493966674039